In [1]:
import psycopg2
import pandas as pd
import numpy as np

conn_string = "host='168.131.30.66' dbname='mimic_iii' user='dbuser' password='jnudl1'"
conn = psycopg2.connect(conn_string)
cur=conn.cursor()

cur.execute("SELECT * FROM capstone.cap_data;")
result = cur.fetchall()

In [2]:
columns = ["row_id","subject_id","hadm_id","intime","outtime","information",'readmission']
df = pd.DataFrame(result,columns = columns)
df.head(10)

,row_id,subject_id,hadm_id,intime,outtime,information,readmission
0,23620,19323,111122,2115-02-04 07:50:48,2115-02-05 07:01:56,[**2115-2-4**] 5:38 AM\n CHEST (PORTABLE AP) ...,0
1,17936,14657,111125,2200-07-05 19:38:11,2200-07-06 23:03:12,[**2200-7-6**] 11:53 AM\n MR L SPINE SCAN ...,0
2,17038,13906,111126,2101-09-01 18:30:04,2101-09-03 18:00:12,[**2101-9-3**] 9:51 AM\n RENAL U.S. ...,0
3,26850,21986,111130,2116-02-25 19:32:29,2116-02-27 18:21:27,[**2116-2-25**] 8:15 PM\n CHEST (PRE-OP PA & L...,0
4,5495,4550,111133,2127-07-12 11:15:51,2127-07-14 16:50:50,[**2127-7-12**] 1:00 PM\n CHEST (PORTABLE AP) ...,0
5,56240,91491,111134,2124-01-28 17:29:03,2124-01-29 13:02:51,Subdural hemorrhage (SDH) s/p crani & evacuati...,0
6,13836,11314,111136,2176-07-23 12:04:22,2176-07-24 15:48:12,[**2176-7-22**] 2:43 PM\n CAROTID SERIES COMPL...,0
7,24562,20122,111138,2140-10-23 22:46:51,2140-10-26 17:34:39,[**2140-10-25**] 6:31 AM\n CHEST (PORTABLE AP)...,0
8,34437,28183,111139,2166-03-30 09:16:43,2166-03-31 13:52:10,"CCU NPN\nS: "" I want to go home.""\nO: See vs/o...",0
9,29259,23942,111141,2189-10-01 11:36:20,2189-10-03 22:07:53,[**2189-10-1**] 3:33 PM\n CHEST (PORTABLE AP) ...,0


In [3]:
import re

def preprocess(x):
    y=re.sub('\\[(.*?)\\]','',x) #remove de-identified brackets
    y=re.sub('[0-9]+\.','',y) #remove 1.2. since the segmenter segments based on this
    y=re.sub('dr\.','doctor',y)
    y=re.sub('m\.d\.','md',y)
    y=re.sub('admission date:','',y)
    y=re.sub('discharge date:','',y)
    y=re.sub('--|__|==','',y)
    # remove, digits, spaces
    result = " ".join(y.split())
    
    return result

In [4]:
total_lst = []
for info in df.iloc[:45942]['information']:
    info = info.split('\n')
    part_lst = []
    for sen in info:
        tmp=preprocess(sen)
        part_lst.append(tmp)
    total_lst.append(part_lst)
    
print(total_lst)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
total = np.array(total_lst)
data = pd.DataFrame(total,columns=["Text"])
print(f'df\'s shape:{df.shape}, data\'s shape:{data.shape}')
new_df = pd.concat([df,data],axis=1)
print(f'new_df\'s shape: {new_df.shape}')
new_df.head()

df's shape:(45942, 7), data's shape:(45942, 1)
new_df's shape: (45942, 8)


C:\Users\170890\Anaconda3\envs\nlp2\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


,row_id,subject_id,hadm_id,intime,outtime,information,readmission,Text
0,23620,19323,111122,2115-02-04 07:50:48,2115-02-05 07:01:56,[**2115-2-4**] 5:38 AM\n CHEST (PORTABLE AP) ...,0,"[5:38 AM, CHEST (PORTABLE AP) Clip #, Reason: ..."
1,17936,14657,111125,2200-07-05 19:38:11,2200-07-06 23:03:12,[**2200-7-6**] 11:53 AM\n MR L SPINE SCAN ...,0,"[11:53 AM, MR L SPINE SCAN Clip #, Reason: wit..."
2,17038,13906,111126,2101-09-01 18:30:04,2101-09-03 18:00:12,[**2101-9-3**] 9:51 AM\n RENAL U.S. ...,0,"[9:51 AM, RENAL U.S. Clip #, Reason: HX RENAL ..."
3,26850,21986,111130,2116-02-25 19:32:29,2116-02-27 18:21:27,[**2116-2-25**] 8:15 PM\n CHEST (PRE-OP PA & L...,0,"[8:15 PM, CHEST (PRE-OP PA & LAT) Clip #, Reas..."
4,5495,4550,111133,2127-07-12 11:15:51,2127-07-14 16:50:50,[**2127-7-12**] 1:00 PM\n CHEST (PORTABLE AP) ...,0,"[1:00 PM, CHEST (PORTABLE AP) Clip #, Reason: ..."


In [6]:
from transformers import DataCollatorWithPadding, AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

tokenizer = AutoTokenizer.from_pretrained("bvanaken/CORe-clinical-mortality-prediction")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained("bvanaken/CORe-clinical-mortality-prediction", num_labels=2)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=evalu,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()